In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/cfrm523/Final_project')
import pandas as pd

Mounted at /content/drive


In [3]:
import numpy as np
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from statsmodels.tsa.vector_ar.vecm import coint_johansen
%matplotlib inline
!pip install arch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.4/983.4 kB 17.6 MB/s eta 0:00:00


In [4]:
from arch.unitroot import ADF
from arch.unitroot import KPSS
from arch.unitroot import PhillipsPerron
from sklearn.linear_model import LinearRegression

In [5]:
#function that checks for stationarity
def stat_testing(df_coint_diff):
  #table_of_stats_diff = {"BTC":[], "ETH":[], "LTC":[], "BCH":[], "XRP":[]}
  table_of_stats_diff = {}

  for i in df_coint_diff.keys():
    table_of_stats_diff[i] = []

  for i in table_of_stats_diff.keys():
    adf = ADF(df_coint_diff[i])
    pp = PhillipsPerron(df_coint_diff[i])
    kpss = KPSS(df_coint_diff[i])

    table_of_stats_diff[i].append(adf.pvalue)
    table_of_stats_diff[i].append(pp.pvalue)
    table_of_stats_diff[i].append(kpss.pvalue)

  df_table_diff = pd.DataFrame.from_dict(table_of_stats_diff)

  tests = ["ADF", "PP", "KPSS"]

  df_table_diff['Test'] = tests
  df_table_diff = df_table_diff.set_index('Test')

  relate = 0
  for i in df_table_diff.keys():
    if df_table_diff[i][0] < 0.05 and df_table_diff[i][1] < 0.05 and df_table_diff[i][2] > 0.10: #if it passes the ADF, PP, KPSS
      relate = 1
      continue
    else:
      relate = False

  return df_table_diff, relate

In [6]:
#creating the spread
def create_coint_df(df, start, end, tickers):
  data_dict = {}
  date_column = None  # To hold the date column

  # Fetch the data for each ticker if available
  for i in tickers:
    data = get_data(df, start, end, i)
    data_dict[i] = data['close']
    if date_column is None:  # Assuming all tickers have the same date range
      date_column = data['date']

  df_coint = pd.DataFrame(data_dict)
  df_coint['date'] = date_column.values
  df_coint.set_index('date', inplace=True)
  df_coint_diff = df_coint.diff().dropna()
  return df_coint_diff


In [7]:
from itertools import combinations

cryptos = ['BTC-USD', 'ETH-USD', 'LTC-USD', 'SOL-USD']
# Collect all combinations from size 2 to the length of the list
all_combinations = []
for r in range(2, len(cryptos)+1):
    all_combinations.extend(list(combinations(cryptos, r)))

spreads = []
# Print all combinations
for combo in all_combinations:
    spreads.append(list(combo))

In [8]:
start = datetime(2024,3,1,0,0,0)
end =   datetime(2024,5,1,0,0,0)
df_ = pd.read_csv("crypto_mar_april.csv")
df_ = df_.drop('Unnamed: 0', axis=1)
df_['date'] = pd.to_datetime(df_['date'])
df = df_.loc[df_['date'] > start]
df = df.loc[df['date'] < end]

def get_data(df, start, end, ticker):
  data = df.loc[df['ticker'] == ticker]
  data_ = data.loc[data['date'] > start]
  data_ = data_.loc[data_['date'] < end]
  data_ = data_.sort_values(by='date', axis=0)
  data_ = data_.drop_duplicates()
  data_ = data_.reset_index(drop=True)

  return data_

In [9]:
um = df.loc[df['date'] >=  datetime(2024, 4, 18, 17, 0, 0)]
um = um.loc[um['date'] <  datetime(2024, 4, 18, 17, 1, 0)]
um = um.drop_duplicates()
print(um)

                      date   ticker      open      high       low     close  \
70140  2024-04-18 17:00:00  BTC-USD  63582.07  63615.36  63551.35  63551.35   
158281 2024-04-18 17:00:00  ETH-USD   3064.34   3064.34   3059.66   3059.66   
246422 2024-04-18 17:00:00  LTC-USD     81.34     81.34     81.23     81.23   
334567 2024-04-18 17:00:00  XRP-USD      0.50      0.50      0.50      0.50   
422718 2024-04-18 17:00:00  SOL-USD    139.48    139.50    139.15    139.15   

              volume  
70140       6.785299  
158281    119.935004  
246422    184.086396  
334567  38196.916500  
422718   1230.108221  


In [10]:
meo = create_coint_df(df, datetime(2024, 4, 18, 17, 0, 0), datetime(2024, 4, 18, 17, 1, 0), spreads[1])

In [11]:
from datetime import datetime, timedelta

#coint_count = {'date':[],'04:00':[], '08:00':[], '12:00':[], '16:00':[], '20:00':[], '00:00':[],'spread':[]}
coint_count = {'date':[],'02:00':[], '04:00':[], '06:00':[], '08:00':[], '10:00':[], '12:00':[], '14:00':[], '16:00':[], '18:00':[], '20:00':[],
                '22:00':[], '00:00':[], 'spread':[]}

time_increment = 2
increment = timedelta(hours=time_increment)
for j in range(len(spreads)):
  start_date = datetime(2024, 3, 1, 0, 0, 0)
  for i in range(60*12): #6 iterations are one day
      end_date = start_date + increment
      if i % 12 == 0:
        coint_count['date'].append(start_date.date())
        coint_count['spread'].append('Spread_'+str(j+1))
      try:
        df_coint_diff = create_coint_df(df, start_date, end_date, spreads[j])
        stuff, e = stat_testing(df_coint_diff)
        formatted_time = end_date.strftime("%H:%M")
        coint_count[formatted_time].append(e)
        #print("Interval", i+1, "from", start_date, "to", end_date)
        # Update start_date to the next interval
        start_date = end_date
      except:
        print(j, i, coint_count['date'][-1])

In [12]:
results = pd.DataFrame.from_dict(coint_count)
results = results.set_index(['spread', 'date'])
pd.set_option('display.max_rows', None)      # Shows all rows
pd.set_option('display.max_columns', None)   # Shows all columns
pd.set_option('display.width', None)         # Auto-detects the width of the display for wrapping
pd.set_option('display.max_colwidth', None)  # Shows full width of columns
results

02:00  04:00  06:00  08:00  10:00  12:00  14:00  16:00  18:00  20:00  22:00  \
spread    date                                                                                      
Spread_1  2024-03-01      1      1      1  False  False      1      1      1      1      1      1   
          2024-03-02      1      1      1      1      1      1  False      1      1      1      1   
          2024-03-03      1  False  False      1      1      1      1      1  False      1      1   
          2024-03-04      1      1      1      1      1      1      1      1      1      1      1   
          2024-03-05      1  False      1  False      1      1      1      1  False      1      1   
          2024-03-06  False      1      1      1  False      1      1      1      1      1      1   
          2024-03-07      1      1      1      1      1      1      1      1      1      1  False   
          2024-03-08  False      1      1      1      1      1      1      1      1      1      1   
          2024-03-09      1      1      1  False      1      1      1      1      1      1      1   
          2024-03-10      1      1      1      1      1      1      1      1      1      1      1   
          2024-03-11      1      1      1      1      1      1  False      1      1      1      1   
          2024-03-12      1      1      1      1  False      1      1      1  False      1      1   
          2024-03-13      1      1      1      1      1      1      1      1      1      1      1   
          2024-03-14      1      1      1      1      1      1      1      1      1      1      1   
          2024-03-15  False      1      1      1      1      1      1      1      1  False      1   
          2024-03-16  False      1      1      1      1      1      1      1      1      1      1   
          2024-03-17      1      1      1  False      1      1      1      1      1      1      1   
          2024-03-18      1      1      1      1      1      1      1      1      1      1      1   
          2024-03-19      1      1      1      1  False  False      1  False      1      1      1   
          2024-03-20      1  False      1      1      1      1      1  False      1      1      1   
          2024-03-21      1      1  False      1  False      1      1      1      1  False      1   
          2024-03-22      1      1      1      1      1      1  False      1      1      1      1   
          2024-03-23      1      1      1      1  False      1      1      1      1      1      1   
          2024-03-24      1      1      1      1      1      1      1      1      1      1      1   
          2024-03-25      1      1      1      1      1      1      1      1      1      1      1   
          2024-03-26      1      1      1  False      1      1      1      1  False      1      1   
          2024-03-27      1      1      1  False      1      1      1      1      1      1  False   
          2024-03-28      1      1      1      1      1      1      1      1      1      1      1   
          2024-03-29      1      1  False  False      1      1      1  False      1      1      1   
          2024-03-30      1      1      1      1      1      1      1      1      1      1      1   
          2024-03-31      1      1      1      1      1      1      1      1      1      1      1   
          2024-04-01      1      1  False  False      1      1      1      1      1      1      1   
          2024-04-02      1  False      1  False      1      1      1  False      1      1      1   
          2024-04-03      1      1      1      1      1      1      1      1      1      1      1   
          2024-04-04      1      1      1      1      1      1      1  False      1      1      1   
          2024-04-05      1  False      1      1      1      1      1      1      1  False  False   
          2024-04-06      1      1      1      1      1      1      1      1      1      1      1   
          2024-04-07      1      1      1      1      1      1      1      1      1

In [13]:
stuff = results.sum()
total = stuff.sort_values(ascending=False)
total

06:00    610
10:00    610
12:00    610
02:00    606
14:00    605
16:00    601
18:00    597
22:00    590
20:00    576
00:00    573
04:00    560
08:00    555
dtype: object

#Total
06:00    610\
10:00    610\
12:00    610\
02:00    606\
14:00    605\
16:00    601\
18:00    597\
22:00    590\
20:00    576\
00:00    573\
04:00    560\
08:00    555\
dtype: object

In [12]:
#month of april

10:00    320
14:00    312
12:00    310
18:00    309
00:00    300
06:00    299
22:00    299
16:00    298
02:00    286
20:00    268
04:00    266
08:00    266
dtype: object

In [ ]:
april

In [ ]:
#month of march
#stuff = results.sum()
#stuff.sort_values(ascending=False)

## March
06:00    311\
02:00    309\
16:00    303\
20:00    301\
12:00    300\
04:00    293\
14:00    293\
22:00    291\
18:00    288\
10:00    287\
08:00    286\
00:00    273\
dtype: object